# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV 
import numpy as np


In [2]:
#для удобсва заказчика будем делать маркировку не 0 1 2, а 1, 2, 3
df_1 = pd.read_csv('/Users/aleksandrivanov/Downloads/geo_data_0.csv')
df_2 = pd.read_csv('/Users/aleksandrivanov/Downloads/geo_data_1.csv')
df_3 = pd.read_csv('/Users/aleksandrivanov/Downloads/geo_data_2.csv')
state = np.random.RandomState(12345)

In [3]:
display(df_1)
display(df_2)
display(df_3)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [4]:
print(df_1.info())
print()
print(df_2.info())
print()
print(df_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column

In [5]:
print(df_1.duplicated().sum())
print(df_2.duplicated().sum())
print(df_3.duplicated().sum())

0
0
0


In [6]:
display(df_1.describe())
print()
display(df_2.describe())
print()
display(df_3.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [7]:
print(df_1.isnull().sum())
print()
print(df_2.isnull().sum())
print()
print(df_3.isnull().sum())
print()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64



In [8]:
df_1['id'].value_counts()

fiKDv    2
QcMuo    2
AGS9W    2
Tdehs    2
74z30    2
        ..
x8osI    1
gZ7tR    1
Zw8hj    1
zcrQO    1
1CWhH    1
Name: id, Length: 99990, dtype: int64

In [9]:
df_1[df_1.duplicated(['id'])]

,id,f0,f1,f2,product
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
66136,74z30,1.084962,-0.312358,6.990771,127.643327
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
75715,Tdehs,0.112079,0.430296,3.218993,60.964018
90815,fiKDv,0.049883,0.841313,6.394613,137.346586
92341,TtcGQ,0.110711,1.022689,0.911381,101.318008
97785,bsk9y,0.378429,0.005837,0.160827,160.637302


In [10]:
df_1[df_1['id'] == 'HZww2']

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695


In [11]:
df_2[df_2.duplicated(['id'])]

,id,f0,f1,f2,product
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [12]:
df_2[df_2['id'] == 'LHZR0']

,id,f0,f1,f2,product
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625


In [13]:
df_3[df_3.duplicated(['id'])]

,id,f0,f1,f2,product
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


In [14]:
df_3[df_3['id'] == 'xCHr8']

,id,f0,f1,f2,product
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641


 Можно удалить столбик с "Id"б он не несет смысловой нагрузкид ля обучения.

In [15]:
df_1 = df_1.drop('id', axis= 1)
df_2 = df_2.drop('id', axis= 1)
df_3 = df_3.drop('id', axis= 1)

Посмотрим как таблица ведет себя по корреляции

In [16]:
df_1.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [17]:
df_2.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [18]:
df_3.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Итоговый вывод:
Все 3 таблицы по структуре одинаковые

Явных выборосов нет, но лучше уточнить на самом деле у бизнеса, реально ли может быть такой разброс

Есть неянвые дубликаты. Возможно, делали в замер в начале эксптуатации скважины и после выхода на режим, удалять ничего пока не буду, так же можно поговорить с бизнесом по этому вопросу

пропуски в данных отсутствуют

Можно удалить столбик с "Id"б он не несет смысловой нагрузкид ля обучения.

По первой таблице( если считать с 0 или по второму месторождению) ситуация отличается, корреляция почти 100% между признаком f2 и продуктом, в остальных 2х она в 2 раза ниже.

То есть добыча во втором датасете очень сильно зависит от одного из признаков

## Обучение и проверка модели

In [19]:
def oil_predict (df):
    target = df['product']
    features = df.drop('product', axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target,
                                                                                  test_size=.25,
                                                                                  random_state=12345)
    reg = LinearRegression()
    parameters = {} #гриду для линейной регрессии не нужны доп параметры, 
    #там всё "зашито" внутри по умолчанию, либо настраивать их вручну.
    grid = GridSearchCV(reg, param_grid = parameters, n_jobs = -1)
    model = grid.fit(features_train,target_train) #обучаем
    predictions_valid = pd.Series(model.predict(features_valid)) #проверяем
    print('Среднее значение сырья по факту: ',round(target_valid.mean(),2)) #оставим 2 цифры после запятой
    print('Среднее значения сырья по предсказанию: ',round(predictions_valid.mean(),2)) #выводим среднее
    print('RMSE: ',mean_squared_error(target_valid,predictions_valid)**0.5) # Считаем показатель RMSE
    
    return predictions_valid, target_valid

In [20]:
predictions_valid_1, target_valid_1 = oil_predict(df_1)

Среднее значение сырья по факту:  92.08
Среднее значения сырья по предсказанию:  92.59
RMSE:  37.5794217150813


In [21]:
predictions_valid_2, target_valid_2 = oil_predict(df_2)

Среднее значение сырья по факту:  68.72
Среднее значения сырья по предсказанию:  68.73
RMSE:  0.8930992867756165


In [22]:
predictions_valid_3, target_valid_3 = oil_predict(df_3)

Среднее значение сырья по факту:  94.88
Среднее значения сырья по предсказанию:  94.97
RMSE:  40.02970873393434


In [23]:

print(predictions_valid_1.shape)
print(target_valid_1 .shape)
print()
print(predictions_valid_2.shape)
print(target_valid_2 .shape)
print()
print(predictions_valid_3.shape)
print(target_valid_3 .shape)

(25000,)
(25000,)

(25000,)
(25000,)

(25000,)
(25000,)


Первые предварительные выводы: самые маленькие запасы получаются у второго месторождения, но у неё самое низкое значение RMSE 

In [25]:
## Подготовка к расчёту прибыли

MONEY = 10000000000
BAR_1000 = 450000
LOSS = 0.025

WELLS = 200
ONE_WELL = MONEY/WELLS
print('Затраты на 1 скважину: ', ONE_WELL)
TARGET_BAR = ONE_WELL/450000
print('требуемое количество с одной скважины для окупаемости, тыс. бар: ', round(TARGET_BAR,2))

Затраты на 1 скважину:  50000000.0
требуемое количество с одной скважины для окупаемости, тыс. бар:  111.11


функция для расчета прибыли

In [26]:
def revenue(target_valid, predictions_model, count):
    
    '''вделим в отдельный датафрейм список правильных ответов target_valid и список предсказаний 
    обученной модели predictions_model, чтобы закончить страдания с перепутавшимися индексами и задублированными значениями''' 
    
    df_rev_calc= pd.DataFrame({'target_valid': target_valid, 'predictions_model': predictions_model})
    
    '''сортируем скважины по убыванию значений предсказаний и вбираем выбираем скважины 
    с максимальными значениями предсказаний, суммируем целевое значение объёма сырья, соответствующее этим максимальным 
    предсказаниям, и считаем прибль для полученного сырья''' 
    
    selected = df_rev_calc.sort_values(by='predictions_model', ascending=False)['target_valid'][:count]

    profit = ((BAR_1000 * selected.sum()) - MONEY)
        
    return profit

In [27]:
print('Первое месторождение:')
print('Итоговый доход:',revenue(target_valid_1.reset_index(drop= True), pd.Series(predictions_valid_1), 200))


Первое месторождение:
Итоговый доход: 3320826043.1398506


In [28]:
print('Второе месторождение:')
print('Итоговый доход:',revenue(target_valid_2.reset_index(drop= True), pd.Series(predictions_valid_2), 200))

Второе месторождение:
Итоговый доход: 2415086696.681511


In [29]:
print('Третье месторождение:')
print('Итоговый доход:',revenue(target_valid_3.reset_index(drop= True), pd.Series(predictions_valid_3), 200))

Третье месторождение:
Итоговый доход: 2710349963.5998325


Во втором месторождении получается самый низкий итоговый доход, лидирует первое месторождение по доходу

## Расчёт прибыли и рисков 

In [30]:
def boot(target, predictions):  
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        predictions_subsample = predictions[target_subsample.index]
        values.append(revenue(target_subsample, predictions_subsample, 200))
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print('Ожидаемая прибыль, млрд рублей:',values.mean()/ 10**9) #посчитаем среднее значение, переведем в млрд для удобвства восприятие
    print('верхняя граница доверительного интервала:', upper)
    print('нижняя граница доверительного интервала:', lower)
    print('Риск потерь',(values < 0).mean() * 100) #переведем в %

In [31]:
print('Первое месторождение:')
boot(target_valid_1.reset_index(drop= True), pd.Series(predictions_valid_1))

Первое месторождение:
Ожидаемая прибыль, млрд рублей: 0.3961649848023711
верхняя граница доверительного интервала: 909766941.5534226
нижняя граница доверительного интервала: -111215545.89049526
Риск потерь 6.9


In [32]:
print('Второе месторождение:')
boot(target_valid_2.reset_index(drop= True), pd.Series(predictions_valid_2))

Второе месторождение:
Ожидаемая прибыль, млрд рублей: 0.4611558172772397
верхняя граница доверительного интервала: 862952060.2637234
нижняя граница доверительного интервала: 78050810.7517417
Риск потерь 0.7000000000000001


In [33]:
print('Третье месторождение:')
boot(target_valid_3.reset_index(drop= True), pd.Series(predictions_valid_3))

Третье месторождение:
Ожидаемая прибыль, млрд рублей: 0.39295047517060444
верхняя граница доверительного интервала: 934562914.5511636
нижняя граница доверительного интервала: -112227625.37857565
Риск потерь 6.5


Вывод

Шансы прогореть на 3 месторождении очень высокий, при этом прибыль меньше, чем  у мервого

Раньше первое месторождение лидировало, но как показал расчет с помощью процедуры Bootstrap - шанс прогореть до 6.5%, мы не проходим по требуемому тз, поэтому откидываем его

Итог: победило месторождение номер 2(датасет 1), там самый маленький запас по нефти, но шанс прогореть до 1%, в результат месторождение получилось хорошим среднячком с минимальными рисками. Если заказчик захочет рискнуть, то может отредактировать тз по риску и выбрать 1 месторождение. 